In [ ]:
import subprocess
import os
import pandas as pd

def ucitaj_sekvencu_iz_fajla(fajl_putanja):
    """
    Učitava sekvencu iz FASTA formata fajla, uklanja zaglavlje i vraća sekvencu kao string.
    """
    with open(fajl_putanja, 'r') as fajl:
        linije = fajl.readlines()
    
    sekvenca = "".join([linija.strip() for linija in linije[1:]])  # Početni red je zaglavlje
    return sekvenca

def generisi_fasta_za_prozor(prozor, indeks):
    """
    Generiše FASTA fajl za svaki prozor.
    """
    fasta_putanja = f"Prozori/prozor_{indeks}.fasta"
    with open(fasta_putanja, 'w') as f:
        f.write(f">prozor_{indeks}\n{prozor}")
    return fasta_putanja

def pokreni_isunstruct_na_fajlu(fajl_putanja):
    """
    Pokreće ISUNSTRUCT na datom FASTA fajlu i vraća njegov izlaz.
    """
    rezultati = subprocess.run(
        ['/home/user/Desktop/IsUnstruct_2.02/IsUnstruct', fajl_putanja],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    print(f"ISUNSTRUCT stdout za {fajl_putanja}:\n{rezultati.stdout}")
    print(f"ISUNSTRUCT stderr za {fajl_putanja}:\n{rezultati.stderr}")
    
    if rezultati.stderr:
        print("*** GREŠKA u izvršavanju ISUNSTRUCT! ***")
    
    return rezultati.stdout

def parsiraj_isunstruct(isunstruct_izlaz):
    """
    Parsira izlaz iz ISUNSTRUCT programa i klasifikuje svaku poziciju u sekvenci kao 'uređenu' ili 'neuređenu'.
    """
    rezultati = []
    for linija in isunstruct_izlaz.splitlines():
        if linija.strip() and not linija.startswith("#"):
            delovi = linija.split()
            if len(delovi) >= 3:
                pozicija = int(delovi[0])
                aminokiselina = delovi[1]
                skor = float(delovi[2])
                klasifikacija = "uređena" if skor <= 0.5 else "neuređena"
                rezultati.append((pozicija, aminokiselina, skor, klasifikacija))
    return rezultati

def generisi_prozore_i_klasifikuj(sekvenca, min_duzina=5, max_duzina=100):
    prozori = []
    oznake_prozora = []
    duzine = []
    pozicije = []
    
    for duzina in range(min_duzina, max_duzina + 1):
        for i in range(len(sekvenca) - duzina + 1):
            podsekvenca = sekvenca[i:i + duzina]
            pozicija = f"{i+1}-{i+duzina}"
            duzine.append(duzina)
            pozicije.append(pozicija)
            
            fasta_putanja = generisi_fasta_za_prozor(podsekvenca, i)
            isunstruct_izlaz = pokreni_isunstruct_na_fajlu(fasta_putanja)
            isunstruct_parsirani_rezultati = parsiraj_isunstruct(isunstruct_izlaz)
            
            podsekvenca_oznake = [result[3] for result in isunstruct_parsirani_rezultati]
            
            if podsekvenca_oznake.count("uređena") > len(podsekvenca_oznake) / 2:
                oznake_prozora.append("uređena")
            else:
                oznake_prozora.append("neuređena")
            
            prozori.append(podsekvenca)
    
    return duzine, pozicije, prozori, oznake_prozora

# Učitavanje sekvence
fajl_putanja = "YP_138523.1.fasta"
sekvenca = ucitaj_sekvencu_iz_fajla(fajl_putanja)

duzine, pozicije, prozori, oznake_prozora = generisi_prozore_i_klasifikuj(sekvenca)

df_sekvence = pd.DataFrame({
    "Duzina": duzine,
    "Pozicija": pozicije,
    "Sekvenca": prozori,
    "ISUNSTRUCT oznaka": oznake_prozora
})

print(df_sekvence.head())
df_sekvence.to_csv("Klasifikovane_sekvence_ISUNSTRUCT.csv", index=False)


ISUNSTRUCT stdout za Prozori/prozor_0.fasta:

ISUNSTRUCT stderr za Prozori/prozor_0.fasta:

ISUNSTRUCT stdout za Prozori/prozor_1.fasta:

ISUNSTRUCT stderr za Prozori/prozor_1.fasta:

ISUNSTRUCT stdout za Prozori/prozor_2.fasta:

ISUNSTRUCT stderr za Prozori/prozor_2.fasta:

ISUNSTRUCT stdout za Prozori/prozor_3.fasta:

ISUNSTRUCT stderr za Prozori/prozor_3.fasta:

ISUNSTRUCT stdout za Prozori/prozor_4.fasta:

ISUNSTRUCT stderr za Prozori/prozor_4.fasta:

ISUNSTRUCT stdout za Prozori/prozor_5.fasta:

ISUNSTRUCT stderr za Prozori/prozor_5.fasta:

ISUNSTRUCT stdout za Prozori/prozor_6.fasta:

ISUNSTRUCT stderr za Prozori/prozor_6.fasta:

ISUNSTRUCT stdout za Prozori/prozor_7.fasta:

ISUNSTRUCT stderr za Prozori/prozor_7.fasta:

ISUNSTRUCT stdout za Prozori/prozor_8.fasta:

ISUNSTRUCT stderr za Prozori/prozor_8.fasta:

ISUNSTRUCT stdout za Prozori/prozor_9.fasta:

ISUNSTRUCT stderr za Prozori/prozor_9.fasta:

ISUNSTRUCT stdout za Prozori/prozor_10.fasta:

ISUNSTRUCT stderr za Prozori/proz